In [161]:
import pandas as pd
import re

In [162]:
class TOP3:
    time    :str
    top3   :list
    def __init__(self, time, top3):
        self.time = time
        self.top3 = top3

In [163]:
def AppendData(dataList:list, sp:list):
    hasKey = False
    for data in dataList:
        if(data[0] == sp[0]):
            data[1] += int(sp[1])
            data[2] += (float(sp[2]) / 1000)
            data[3] += float(sp[4])
            data[4] += float(sp[6])
            return 


    if not hasKey:
            row = []
            row.append(sp[0])
            row.append(int(sp[1]))
            row.append(float(sp[2]) / 1000)
            row.append(float(sp[4]))
            row.append(float(sp[6]))
            dataList.append(row)

In [164]:
def parseFunc(time, funcLines:list):
    threadCnt = 0
    header = ['Function','Hit', 'Time(ms)', 'Avg(us)', 's^2(us)']
    dataList = []
    top10 = []
    df = pd.DataFrame()

    for line in funcLines:
        if '==>' in line:
            threadCnt += 1
            continue
        if (len(line) == 93):
            AppendData(dataList, line.split())
        else:
            continue
    
    df = pd.DataFrame(data = dataList, columns = header)
    totalTime = df[header[2]].sum()
    df['time_ratio(%)'] = round(df[header[2]]/totalTime * 100, 1)
    df[header[2]] = round(df[header[2]], 3)
    df[header[3]] = round(df[header[3]] / threadCnt, 3)
    df[header[4]] = round(df[header[4]] / threadCnt, 3)

    # Get Top3
    df = df.sort_values(by=['time_ratio(%)'], ascending=False).head(3)
    top3 = []
    for idx, row in df.loc[:,['Function', 'time_ratio(%)']].iterrows():
        top3.append(f'{row["Function"]} ({row["time_ratio(%)"]}%)')
    return TOP3(time, top3)
    ## return top 3 func list

In [165]:

filePath = './raw/trace_func.txt'
f = open(filePath)

time = 0
lines = []
TopData = []

## 1초 단위로 lines 모아서 parseFunc 으로 전달
for line in f.readlines():
    if line.startswith('Time:'):
        if len(lines) > 1:
            TopData.append(parseFunc(time, lines))
        time = re.findall(r'Time:\s([\d.]+)\s', line)[0]
        time = int(round(float(time)))
        lines.clear()
    lines.append(line)

TopData.append(parseFunc(time, lines))
f.close()

outfile = open(filePath + '.parse', 'w')
outfile.write('time,top1,top2,top3\n')
for top3 in TopData:
    outfile.write(f'{top3.time},{",".join(top3.top3)}\n')
outfile.close()

